In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../Data/www/styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
import requests 
import requests.auth
import json
#from PIL import Image
#from io import BytesIO
from IPython.display import display, Image


## Using Reddit's API

Reddit is an entertainment, social news networking service, and news website. Reddit's registered community members can submit content, such as text posts or direct links. Registered users can then vote submissions up or down to organize the posts and determine their position on the site's pages. The submissions with the most positive votes appear on the main page or the top of a category. 

Content entries are organized by areas of interest called "subreddits". The subreddit topics include news, gaming, movies, music, books, fitness, food, and photosharing, among many others.  

As you would expect, this is the website where you can find amazing information, such as 

<img width='600' style="float:center" src='../Data/www/images/reddit.png')></img>

enabling you to watch wonderful animated gifs:

<img width='300' style="float:center" src='http://i.imgur.com/hls96Kv.gif'></img>

Of course, what really get users of Reddit to come back are the [comments](https://www.reddit.com/r/aww/comments/4v00p4/owls_are_just_cats_with_wings_think_about_it/). 

+++

But enough of this. 

Reddit's API creates a new challenge for us.  In order to use it, we need to have an account and we need to **authenticate** our requests to the API.

We are going to be doing a couple of different things here in order to authenticate our interactions with the API.  First, we will create a session. This is done using cookies. Fortunately, we do not need to worry about any of those details because `requests` will take care of it for us.

Next, we will login to the reddit API and obtain a `modhash`. A `modhash` is a token that the reddit API requires to help prevent `Cross-Site Request Forgery` (CSRF), that is, an attack that forces an end user to execute unwanted actions on a web application in which they're currently authenticated. 

The following code was taken from https://github.com/reddit/reddit/wiki/OAuth2-Quick-Start-Example

In [3]:
# app info
app_name = 'nico101'
redirect_uri = 'http://www.example.com/unused/redirect/uri'
app_developers = ['lamaral1968']

#set username and password values NONE OF THESE SHOULD BE HERE
username = 'lamaral1968'
password = 'TT939wKoCgeF69BBQkA8'
app_client_ID = 'QUOj7JZLVBh-lQ'
app_client_secret = 'mIZyCYYE9uD4RB07-GJ1Hgs2AGI'             


client_auth = requests.auth.HTTPBasicAuth(app_client_ID, app_client_secret) 
post_data = {"grant_type": "password", "username": username, "password": password}
headers = {"User-Agent": "NICO_bot/0.1 by lamaral1968"}
response = requests.post("https://www.reddit.com/api/v1/access_token", 
                         auth = client_auth, data = post_data, headers = headers)
response.json()


{'access_token': 'nqP5B8ZcgHYmyBhAC66NC_D4SzI',
 'expires_in': 3600,
 'scope': '*',
 'token_type': 'bearer'}

In [4]:
access_token = response.json()['access_token']
headers.update( {'Authorization': 'bearer ' + access_token } )
response = requests.get("https://oauth.reddit.com/api/v1/me", headers = headers)
response.json()

{'comment_karma': 0,
 'created': 1469750771.0,
 'created_utc': 1469721971.0,
 'features': {'activity_service_read': True,
  'activity_service_write': True,
  'adblock_test': True,
  'ads_auction': True,
  'ads_auto_extend': True,
  'ads_auto_refund': True,
  'adserver_reporting': True,
  'adzerk_do_not_track': True,
  'adzerk_reporting_2': True,
  'do_not_track': True,
  'eu_cookie_policy': True,
  'expando_events': True,
  'force_https': True,
  'give_hsts_grants': True,
  'https_redirect': True,
  'image_uploads': True,
  'imgur_gif_conversion': True,
  'legacy_search_pref': True,
  'live_happening_now': True,
  'moat_tracking': True,
  'mobile_native_banner': True,
  'mobile_settings': True,
  'mobile_web_targeting': True,
  'new_loggedin_cache_policy': True,
  'new_report_dialog': True,
  'orangereds_as_emails': True,
  'outbound_clicktracking': True,
  'pause_ads': True,
  'pokemongo_content': {'experiment_id': 47, 'variant': 'control_2'},
  'post_embed': True,
  'screenview_event

If all we want to do is to search Reddit for content, then we do not need to authenticate.  Let's say that we want to look for posts to Reddit that feature puppies.  How would we go about doing it?

In [5]:
reddit_query = 'http://www.reddit.com/r/puppies/search.json'
options = {'q': 'adorable', 'sort': 'new', 'restrict_sr': 'on'}

response = requests.get( reddit_query, params = options )
print(response.url)
response.json()

http://www.reddit.com/r/puppies/search.json?q=adorable&sort=new&restrict_sr=on


{'data': {'after': 't3_3jnsv2',
  'before': None,
  'children': [{'data': {'approved_by': None,
     'archived': False,
     'author': 'jbviewpointz',
     'author_flair_css_class': None,
     'author_flair_text': None,
     'banned_by': None,
     'clicked': False,
     'created': 1471955370.0,
     'created_utc': 1471926570.0,
     'distinguished': None,
     'domain': 'i.reddituploads.com',
     'downs': 0,
     'edited': False,
     'from': None,
     'from_id': None,
     'from_kind': None,
     'gilded': 0,
     'hidden': False,
     'hide_score': False,
     'id': '4z4qbs',
     'is_self': False,
     'likes': None,
     'link_flair_css_class': None,
     'link_flair_text': None,
     'locked': False,
     'media': None,
     'media_embed': {},
     'mod_reports': [],
     'name': 't3_4z4qbs',
     'num_comments': 3,
     'num_reports': None,
     'over_18': False,
     'permalink': '/r/puppies/comments/4z4qbs/guess_he_thinks_nikes_are_comfortable_too_lol/?ref=search_posts',
   

In [6]:
goodies = response.json()
print(type(goodies['data']['children']))
stories = goodies['data']['children']
print(stories[1])

<class 'list'>
{'kind': 't3', 'data': {'author': 'Nick360S', 'hide_score': False, 'name': 't3_4s75x7', 'distinguished': None, 'mod_reports': [], 'thumbnail': 'http://b.thumbs.redditmedia.com/aCp5oTKRWAxjjYS-5w8ZYX0_Z1XQMQ2iHmPXYDHlQ2w.jpg', 'edited': False, 'locked': False, 'num_reports': None, 'user_reports': [], 'downs': 0, 'subreddit': 'puppies', 'preview': {'images': [{'id': 'ghZKyG8x9A3oaIow_sDyc6pv5tJwNXAqPcth7dvjb50', 'source': {'height': 1080, 'url': 'https://i.redditmedia.com/fGnsJhxJeNby4jUSHCsT4Xd7WAJUgUGRjH88EW9J6sc.jpg?s=41c9a34f70542d2018acb5d23a10bbdd', 'width': 1920}, 'variants': {}, 'resolutions': [{'height': 60, 'url': 'https://i.redditmedia.com/fGnsJhxJeNby4jUSHCsT4Xd7WAJUgUGRjH88EW9J6sc.jpg?fit=crop&amp;crop=faces%2Centropy&amp;arh=2&amp;w=108&amp;s=06933e5c93169a7995ae1c5a8f18e94d', 'width': 108}, {'height': 121, 'url': 'https://i.redditmedia.com/fGnsJhxJeNby4jUSHCsT4Xd7WAJUgUGRjH88EW9J6sc.jpg?fit=crop&amp;crop=faces%2Centropy&amp;arh=2&amp;w=216&amp;s=de317938e836

`['data']['children']` is where the posts are.  We can see that we have a bunch of `keys` storing different types of information for each post.  We can look into more detail into the contents of our stories/posts.

In [7]:
print(stories[1].keys())

dict_keys(['kind', 'data'])


In [8]:
print(stories[0]['kind'])
print(stories[0]['data'].keys())


t3
dict_keys(['author', 'hide_score', 'name', 'distinguished', 'mod_reports', 'thumbnail', 'edited', 'locked', 'num_reports', 'user_reports', 'downs', 'subreddit', 'preview', 'score', 'quarantine', 'archived', 'created', 'permalink', 'id', 'likes', 'selftext', 'is_self', 'domain', 'subreddit_id', 'url', 'stickied', 'media', 'removal_reason', 'author_flair_text', 'post_hint', 'from', 'saved', 'num_comments', 'clicked', 'visited', 'media_embed', 'ups', 'gilded', 'link_flair_text', 'suggested_sort', 'from_kind', 'secure_media_embed', 'title', 'link_flair_css_class', 'secure_media', 'report_reasons', 'hidden', 'selftext_html', 'over_18', 'from_id', 'approved_by', 'banned_by', 'created_utc', 'author_flair_css_class'])


In [9]:
print(stories[0]['data']['author'])
print(stories[0]['data']['id'])
print(stories[0]['data']['score'])
print(stories[0]['data']['thumbnail'])

jbviewpointz
4z4qbs
69
http://b.thumbs.redditmedia.com/gL_3xKGx_4mcY6aLug8ScQJ_276lpAALCGT8HrgYvoI.jpg


An interesting bit of information is that posts with images have URLs stored under the key `thumbnail`. Let's select all posts that have pictures.


In [10]:
for i, story in enumerate(stories):
    if 'http' in story['data']['thumbnail']:
        figure_url = story['data']['thumbnail']
        print(i, story['data']['name'], story['data']['created_utc'])
        display(Image(url=figure_url))

0 t3_4z4qbs 1471926570.0


1 t3_4s75x7 1468181809.0


3 t3_4nmyke 1465675159.0


4 t3_4loi8d 1464594325.0


5 t3_4g5rmf 1461453076.0


...

19 t3_3nqriw 1444158605.0


20 t3_3n3dha 1443701725.0


22 t3_3kzs1l 1442284712.0


23 t3_3kix9j 1441966501.0


24 t3_3jnsv2 1441397044.0


In [11]:
index = 4
print(stories[index]['kind'])
print(stories[index]['data']['id'])
print(stories[index]['data']['score'])
print(stories[index]['data']['name'])
print('http://www.reddit.com/' + stories[index]['data']['id'])

t3
4loi8d
168
t3_4loi8d
http://www.reddit.com/4loi8d


That one is clearly too cute to leave us unmoved. **We must upvote that post**, but how can we do it?

First, it turns out you cannot vote on posts that are more than 6 months old.  All posts are archived after 6 months. So, we need to find a post that is younger than 6 months.  

Fortunately, we've already checked the date and that post is from April 23, 2016.



In [12]:
print(stories[index]['data']['name'])

t3_4loi8d


Next, we need to obtain all relevant information for voting. From the developer pages we learn that: 

    POST /api/vote  Cast a vote on a thing.

    id should be the fullname of the Link or Comment to vote on.

    dir indicates the direction of the vote. Voting 1 is an upvote, -1 is a downvote, and 0 is equivalent to "un-voting" by clicking again on a highlighted arrow.have to obtain the URL of the post.

`fullname` is created by adding the `kind` and the `id`.

In [13]:
name = stories[index]['data']['name']
vote = requests.post('https://oauth.reddit.com/api/vote', headers = headers, params = {'id': name, 'dir': 1} )
print(vote.url)
print(vote.status_code)
print(vote.text)

https://oauth.reddit.com/api/vote?id=t3_4loi8d&dir=1
200
{}


Excellent! We've now voted on that adorable post! We've also successfully `POST`ed data using an API for the first time also. 

While we've had fun exploring this example, all APIs work on these exact same principles (some are just documented better than others) so you're now equipped to work with APIs all over the web!